# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다 
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다
4. A, B와 대응되는 C를 추출한다
5. D를 입력받는다 

=>
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다. 
ex) A: 산, B: 바다, C: 나무, D:물

**<출력 예시>**

관계 [빠리 : 용감하다 = 께선 : ?] <br>
모델이 예측한 가장 적합한 단어 : 잠입 <br>
당신의 답변과 모델 예측의 유사도 : 0.34 <br>
아쉽네요. 더 생각해보세요.

In [ ]:
# import gdown 

# url = 'https://drive.google.com/uc?id=1DCgLPJsfyLGZ99lB-aF8EvpKIWSZYgp4'
# output = 'ted_en.xml'  # ted 연설 정보(데이터 크기가 큼) - xml 데이터라 태그가 포함되어 있음. 

# gdown.download(url, output)

In [15]:
from lxml import etree  # xml, html 을 처리하기 위해 경량화된 라이브러리 - etree : xml 파일을 파싱하고 처리할 수 있게 만들어주는 모듈
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from konlpy.tag import Okt

In [ ]:
# !pip install korpora

  Using cached xlrd-2.0.2-py2.py3-none-any.whl.metadata (3.5 kB)
Using cached xlrd-2.0.2-py2.py3-none-any.whl (96 kB)

   -------------------------- ------------- 2/3 [korpora]
   ---------------------------------------- 3/3 [korpora]



In [9]:
import pandas as pd

sent = pd.read_csv("kor_sentences.tsv", sep="\t", header=None, names=["id","lang","text"])  # Tatoeba 한국어 문장 덤프 데이터 
ko = sent[sent["lang"]=="kor"][["text"]]
ko.head()

,text
0,뭔가 해보자!
1,자야 합니다.
2,뭐해?
3,그것은 무엇입니까?
4,"오늘은 6월 18일, Muiriel의 생일입니다!"


In [ ]:
text = '\n'.join(sent['text'])  # \n(개행) 기준으로 조인하기 
text

'뭔가 해보자!\n자야 합니다.\n뭐해?\n그것은 무엇입니까?\n오늘은 6월 18일, Muiriel의 생일입니다!\n생일 축하해 Muiriel!\nMuiriel은 지금 20살입니다.\n비밀번호는 "Muiriel" 입니다.\n곧 돌아올께요.\n모르겠어요.\n이건 절대 끝나지 않을거야.\n뭐라고 말해야 할지 정말 모르겠어...\n나는 산에 있었어.\n이 사진 최근 거에요?\n시간 있는지 잘 모르겠어요.\n이 세상의 교육이 나를 실망시켜요.\n저는 하루에 100유로를 법니다.\n왜냐하면 당신은 혼자가 되고 싶지 않기 때문이에요.\n보고 싶어.\n당신 좀 자야겠어요.\n나는 갈 거에요.\n유감스럽게도 사실이네요.\n정말이에요?\n일어났을 때 슬펐어.\n혼자인 사람은 다른 사람들을 두려워하기 때문에 혼자다.\n일본에 갈 수 있었으면 좋겠어요.\n사람 많은 게 싫어요.\n자야겠어요.\n오늘은 더 이상 물어보지 않겠어요.\n그가 사과를 했음에도 불구하고, 난 아직 화가 나 있어요.\n아는분이 없을겁니다.\n해결책을 찾았지만 너무 빨리 찾아서 맞는 답이 아닌거 같습니다.\n무엇이 문제입니까?\n더이상 어떻게 해야할지 모르겠어.\n나는 언젠가는 프랑스에 꼭 가야되는데, 그게 언제인지는 모르겠어.\n아무나 할 수 있는 것이 아닙니다.\n모두가 너를 만나고 싶어해, 너 유명하네!\n왜 안되는지 설명할려면 시간 너무 많이 걸리겠다.\n작고 흰 토끼는 "무슨 일 있니?" 라고 물어봤습니다.\n히로시는 이것 재미 있을 것 같다고 말한다.\n우리가 가든지 안 가든지는 네가 결정해.\n저는 남은 인생을 그것에 대해 후회하면서 지내고 싶지 않아요.\n나는 꿈이 있어.\n놀라워 하고, 궁금해 하는 것이 이해의 시작이다.\n그러나 우주는 무한하다.\n한가지만 빼면 그녀는 완벽해요.\n그러나, 그 반대도 또한 항상 진실이다.\n그 모습 그대로 사물을 보지 않고, 우리 자신의 모습으로 그것을 바라본다.\n이 세상은 미치광이들의 소굴이다\n너 멍청하게 보여.\n난 쇼핑 가야되요, 한 시간 안에 돌아

In [27]:
# 데이터 전처리 (토큰화, 대소문자 정규화, 불용어 처리)
# sentences = sent_tokenize(corpus)

# def load_stopwords(filepath):
#     with open(filepath, 'r', encoding='UTF-8') as f :
#         stopwords = [line.strip() for line in f]          # strip : 앞뒤 공백 제거 
#     return stopwords

# preprocess_sentences = []
# ko_stopwords = load_stopwords('ko_stopwords.txt')

# for sentence in sentences:
#     sentence = sentence.lower()
#     sentence = re.sub(r'[^가-헿ㄱ-ㅎㅏ-ㅣ0-9]', ' ', sentence)   # 영소문자, 숫자 제외 제거 (특수문자 제거)
#     tokens = word_tokenize(sentence)
#     tokens = [token for token in tokens if token not in ko_stopwords]
#     preprocess_sentences.append(tokens)

okt = Okt()
morphs = okt.morphs(text)

# nouns = okt.nouns(morphs)
# nouns[10:30]

morphs = okt.morphs(text)            
nouns  = okt.nouns(" ".join(morphs))
nouns[:3]

['뭔가', '뭐', '해']

In [ ]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(
    sentences=preprocess_sentences,
    vector_size=100, 
    window=5, 
    min_count=5, 
    sg=0
)

------

In [32]:
sent_df = pd.read_csv("kor_sentences.tsv", sep="\t", header=None)

sent_df.isnull().sum()
sent_df = sent_df.dropna(how='any')  # 결측치 제거 

In [35]:
sent_df[2][100:300]

100               아마도 너가 옳을지도 몰라, 내가 너무 이기적이었어.
101                    모든 사람들이 두번째 기회를 받아 마땅하다.
102                            이 기술의 이점은 무엇인가요?
103    만약 당신에게 이 프로그램이 없다면, 지금 다운로드 받으실 수 있습니다.
104                    당신을 너무 오래 기다리게 해서 죄송합니다.
                         ...                   
295                              재떨이 하나 가져다주세요.
296                                      너 걱정마.
297                                    이것은 안비싸다
298                             카드로 지불할 수 있습니까?
299                               번개가 나무에 떨어졌다.
Name: 2, Length: 200, dtype: object

In [40]:
sent_df[2] = sent_df[2].replace(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', regex=True)

In [41]:
sent_df[2]

0                                    뭔가 해보자
1                                    자야 합니다
2                                        뭐해
3                                 그것은 무엇입니까
4                           오늘은 월 일 의 생일입니다
                        ...                
14215                 매튜는 아무것도 제대로 볼 수가 없었다
14216               마지막으로 길바닥에서 돈 찾았던 게 언제야
14217                         강이 호수로 흘러들어간다
14218    지원이는 애가 대체 어디를 가서 이렇게 늦게까지 안 오는 거람
14219    학생의 손으로 학생 자치가 무너져가고 있다면 어떻게 해야하나요
Name: 2, Length: 14220, dtype: object

In [67]:
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()

def load_stopwords(filepath):
    with open(filepath, 'r', encoding='UTF-8') as f :
        stopwords = [line.strip() for line in f]          # strip : 앞뒤 공백 제거 
    return stopwords

ko_stopwords = load_stopwords('ko_stopwords.txt')

preprocess_data = []

for sentence in tqdm(sent_df[2]) :   
    tokens = okt.morphs(sentence, stem=True)
    tokens = [token for token in tokens if token not in ko_stopwords]
    tokens = [token for token in tokens if len(token) > 2]
    # nouns = okt.nouns(tokens)
    preprocess_data.append(tokens)

preprocess_data[100:300]

100%|██████████| 14220/14220 [00:09<00:00, 1484.07it/s]


[['아마도', '모르다'],
 ['두번째', '마땅하다'],
 ['인가요'],
 ['프로그램', '다운로드'],
 ['기다리다', '죄송하다'],
 ['이르다', '죄송하다'],
 ['이렇게', '비싸다', '안되다'],
 ['이렇다', '따뜻하다', '비정상'],
 ['안녕하다'],
 [],
 ['이야기', '진지하다'],
 ['나가다'],
 ['버리다'],
 ['내리다', '정류장', '지나치다'],
 [],
 ['이에요'],
 [],
 [],
 [],
 ['보이다'],
 ['만나다'],
 [],
 ['한국말'],
 ['한국어'],
 ['노래방', '마시다'],
 [],
 ['대단하다', '감사하다'],
 ['일어나다'],
 [],
 [],
 ['잃어버리다'],
 ['바쁘다'],
 [],
 [],
 [],
 [],
 ['즐겁다'],
 ['마르다'],
 ['필요하다', '어딘가'],
 [],
 ['시간표'],
 ['올림픽'],
 ['수줍다', '완전하다', '쑥스럽다'],
 [],
 [],
 [],
 [],
 ['주인공'],
 [],
 ['힘드다'],
 [],
 [],
 ['회복하다'],
 ['아프다', '아니다', '그렇다'],
 ['인내심'],
 ['피카소', '드물다'],
 ['성가시다'],
 ['미치다', '라벤더'],
 [],
 ['머무르다'],
 ['보여주다'],
 ['하룻밤', '그다음', '죽이다'],
 [],
 [],
 ['푸에르토리코', '보여주다'],
 [],
 ['발자국', '따라가다'],
 ['라자로', '자멘호픈', '폴란드인', '이에요'],
 ['스타크래프트'],
 ['태어나다', '자유롭다'],
 ['캐비아'],
 ['할아버지', '크리스마스', '여자친구'],
 ['목구멍', '손가락', '찌르다'],
 ['분노하다', '잠자다'],
 ['사나이'],
 [],
 [],
 ['아름답다'],
 ['여보세요'],
 [],
 [],
 [],
 [],
 ['빛나다'],
 ['몰타어', '빠지다'],
 [],
 [],
 ['어렵다'],

In [93]:
# # 각 문장 리스트를 하나의 문자열로 합치기
# preprocess_data_2 = [" ".join(tokens) for tokens in preprocess_data]

# # 이제 전체를 합쳐서 하나의 텍스트로 만들기
# text = " ".join(preprocess_data_2)

# # 명사 추출
# nouns = okt.nouns(text)
# nouns = [n for n in nouns if len(n) > 2]
# print(nouns[10:])  # 앞 10개만 확인
# print(len(nouns))

# nouns_per_sentence = [okt.nouns(" ".join(tokens)) for tokens in preprocess_data]
# nouns_per_sentence[:10]

nouns_per_sentence = [
    [w for w,t in okt.pos(" ".join(s), norm=True, stem=True) if t=='Noun']
    for s in preprocess_data
]
nouns_per_sentence = [ns for ns in nouns_per_sentence if ns]  # 빈 리스트 제거



In [94]:
from gensim.models import FastText

sent_model = FastText(        
    sentences=nouns_per_sentence, 
    vector_size=100, 
    window=5, 
    min_count=5, 
    sg=0
)

In [97]:
sent_model.wv.vectors.shape

(184, 100)

In [95]:
sent_model.wv.most_similar('메세지')

[('한국말', 0.23624110221862793),
 ('에스페란토', 0.22208347916603088),
 ('브라질', 0.22081099450588226),
 ('캐나다인', 0.21362578868865967),
 ('불리', 0.21033945679664612),
 ('달리기', 0.19959671795368195),
 ('캐나다', 0.19583013653755188),
 ('강아지', 0.195212721824646),
 ('박물관', 0.18248583376407623),
 ('과학자', 0.17719733715057373)]

In [96]:
sent_model.wv['메세지']

array([-2.6984038e-04, -4.8525543e-03, -3.4813560e-04, -5.4224255e-04,
        5.0281337e-04,  9.8445290e-04, -2.0835616e-03,  9.8823174e-04,
       -1.6630045e-04, -2.9995700e-03, -2.5606153e-03,  2.0421152e-04,
        3.2142431e-03,  4.2973552e-04, -5.4312437e-03, -4.9726716e-03,
       -8.4618159e-04,  1.3637690e-04, -4.0925276e-03,  7.3349226e-04,
       -1.0310990e-03, -1.7227996e-03,  3.3928137e-03, -8.1268525e-05,
        2.0419925e-03, -1.2974258e-03, -3.5200524e-04, -2.5398310e-03,
        4.2052534e-03,  3.0298493e-04,  4.5609777e-03, -1.0529313e-03,
       -3.0299518e-03, -1.2014842e-03,  1.4446449e-03,  5.1443093e-04,
       -1.8287111e-03,  7.7262521e-06, -2.7321195e-04,  1.7317231e-03,
        3.7716234e-03, -3.5325156e-03, -1.6210485e-03,  1.7331656e-03,
        4.0238369e-03,  3.5338942e-04,  4.1232128e-03,  1.7787585e-03,
        4.2267999e-05,  2.3429303e-03,  2.4399359e-03,  2.0552352e-03,
        1.0650622e-03, -2.8667848e-03,  2.0396330e-04, -2.6340999e-03,
      

In [ ]:
import random
from gensim.models import FastText

model = sent_model  # 이미 학습된 FastText 모델 객체를 사용한다고 가정

# ---------------------------
# 유틸 함수들
# ---------------------------
def is_in_vocab(w):
    return w in model.wv.key_to_index

def random_word(exclude=set(), min_len=2):
    keys = model.wv.key_to_index.keys()
    while True:
        w = random.choice(list(keys))
        if (w not in exclude) and (len(w) >= min_len):
            return w

def sample_pair_with_similarity(threshold=0.8, max_tries=5000):
    """
    유사도 >= threshold인 (A,B) 한 쌍을 무작위로 뽑는다.
    """
    keys = list(model.wv.key_to_index.keys())
    for _ in range(max_tries):
        A = random.choice(keys)
        B = random.choice(keys)
        if A == B:
            continue
        sim = model.wv.similarity(A, B)
        if sim >= threshold:
            return A, B, sim
    raise RuntimeError(f"유사도>={threshold}인 쌍을 찾지 못했습니다. (max_tries={max_tries}): threshold를 낮춰보세요.")

def sample_C(exclude, max_tries=2000):
    """
    A,B와 너무 겹치지 않는 C를 뽑는다.
    """
    keys = list(model.wv.key_to_index.keys())
    for _ in range(max_tries):
        C = random.choice(keys)
        if C not in exclude and len(C) >= 2:
            return C
    raise RuntimeError("C를 뽑지 못했습니다.")

def compute_D_candidates(A, B, C, topn=10):
    """
    벡터 연산 B - A + C로 D 후보들을 구한다.
    A,B,C 자체는 제외.
    """
    candidates = model.wv.most_similar(positive=[B, C], negative=[A], topn=topn)
    filtered = [(w, s) for w, s in candidates if w not in {A, B, C}]
    return filtered

# ---------------------------
# 게임 본체
# ---------------------------
def play_analogy_game(rounds=5, sim_threshold=0.8, topn=10):
    print("== 단어 유추 게임 시작 ==")
    print("형식:  A : B  =  C : D  → D를 맞혀보세요!\n")
    score = 0

    for r in range(1, rounds+1):
        # 1) (A,B) 샘플링 (유사도 >= threshold)
        try:
            A, B, ab_sim = sample_pair_with_similarity(threshold=sim_threshold)
        except RuntimeError as e:
            print(e)
            break

        # 2) C 샘플링
        C = sample_C(exclude={A, B})

        # 3) D 후보 계산
        D_candidates = compute_D_candidates(A, B, C, topn=topn)
        if not D_candidates:
            # 후보가 없다면 라운드 재시도
            # (데이터가 작거나 희귀어가 많을 때 발생 가능)
            continue
        D_true, D_score = D_candidates[0]  # 정답 후보 1위

        # 4) 문제 제시
        print(f"[Round {r}]  {A} : {B}  =  {C} : ?")
        guess = input("당신의 답 D: ").strip()

        # 5) 판정
        correct_set = {w for w, _ in D_candidates[:3]}  # 상위 3개까지 정답 인정(난이도 조절)
        if guess in correct_set:
            print(f"✅ 정답! (정답 후보: {D_true}, 상위 후보: {', '.join(correct_set)})\n")
            score += 1
        else:
            print(f"❌ 오답!  정답 후보: {D_true}")
            print(f"   상위 후보들: {', '.join([w for w,_ in D_candidates[:5]])}\n")

    print(f"== 게임 종료: 점수 {score} / {rounds} ==")



In [101]:
# 실행
play_analogy_game(rounds=5, sim_threshold=0.1, topn=10)

== 단어 유추 게임 시작 ==
형식:  A : B  =  C : D  → D를 맞혀보세요!

[Round 1]  화장실 : 마르다  =  노래방 : ?
❌ 오답!  정답 후보: 아프리카
   상위 후보들: 아프리카, 일본인, 쓰레기, 월요일, 동물원

[Round 2]  알코올 : 설거지  =  스페인 : ?
❌ 오답!  정답 후보: 프랑스
   상위 후보들: 프랑스, 한국말, 데리다, 월요일, 스포츠

[Round 3]  냉장고 : 드레스  =  인터넷 : ?
❌ 오답!  정답 후보: 바나나
   상위 후보들: 바나나, 아파트, 멕시코, 아프리카, 제대로

[Round 4]  어디가 : 비행기  =  어려움 : ?
❌ 오답!  정답 후보: 자폐증
   상위 후보들: 자폐증, 러시아어, 자전거, 원숭이, 한국말

[Round 5]  가까이 : 슈퍼마켓  =  스트레스 : ?
❌ 오답!  정답 후보: 일요일
   상위 후보들: 일요일, 비디오, 할아버지, 보스턴, 대학생

== 게임 종료: 점수 0 / 5 ==


-----

wiki 데이터

In [1]:
from Korpora import Korpora
Korpora.fetch("namuwikitext", root_dir='wiki.text')

[Korpora] Corpus `namuwikitext` is already installed at c:\skn_17\nlp\03_word_embedding\wiki.text\namuwikitext\namuwikitext_20200302.train.zip
[Korpora] Corpus `namuwikitext` is already installed at c:\skn_17\nlp\03_word_embedding\wiki.text\namuwikitext\namuwikitext_20200302.train
[Korpora] Corpus `namuwikitext` is already installed at c:\skn_17\nlp\03_word_embedding\wiki.text\namuwikitext\namuwikitext_20200302.test.zip
[Korpora] Corpus `namuwikitext` is already installed at c:\skn_17\nlp\03_word_embedding\wiki.text\namuwikitext\namuwikitext_20200302.test
[Korpora] Corpus `namuwikitext` is already installed at c:\skn_17\nlp\03_word_embedding\wiki.text\namuwikitext\namuwikitext_20200302.dev.zip
[Korpora] Corpus `namuwikitext` is already installed at c:\skn_17\nlp\03_word_embedding\wiki.text\namuwikitext\namuwikitext_20200302.dev


In [2]:
import pandas as pd

wiki = pd.read_csv("./wiki.text/namuwikitext/namuwikitext_20200302.dev", sep="\t", header=None)
wiki.head()
# len(wiki)

,0
0,= 에일리언: 컨테인먼트 =
1,Alien: Containment. 에이리언 1편 40주년을 맞아 2019년 3월에...
2,참고로 우연의 일치인지 컨테인먼트라는 단어 자체가 에이리언: 커버넌트와 유사하다.
3,= Exiled Kingdoms =
4,= = 개요 = =


In [3]:
wiki[0].isnull().sum()

0

In [4]:
wiki[0] = wiki[0].replace(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', regex=True)

In [5]:
wiki[0]

0                                               에일리언 컨테인먼트 
1           에이리언 편 주년을 맞아 년 월에 공개된 분 초짜리 단편으로 목성형 행성을 지나...
2               참고로 우연의 일치인지 컨테인먼트라는 단어 자체가 에이리언 커버넌트와 유사하다
3                                                          
4                                                      개요  
                                ...                        
117992    합체 시 조종석은 한 군데로 모이지 않으며 일괄적으로 조종하는 것이 아닌 각 부위에...
117993    현재까지 수많은 전투를 벌였지만 단 한번도 패배한 적이 없는 무적의 메카이다 이는 ...
117994                                              같이 보기  
117995                                    역대 자유 인도 임시정부 주석 
117996                                                     
Name: 0, Length: 117997, dtype: object

In [ ]:
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()

def load_stopwords(filepath):
    with open(filepath, 'r', encoding='UTF-8') as f :
        stopwords = [line.strip() for line in f]          # strip : 앞뒤 공백 제거 
    return stopwords

ko_stopwords = load_stopwords('ko_stopwords.txt')

preprocess_data = []

for sentence in tqdm(wiki[0]) :   
    tokens = okt.morphs(sentence, stem=True)
    tokens = [token for token in tokens if token not in ko_stopwords]
    tokens = [token for token in tokens if len(token) > 2]
    preprocess_data.append(tokens)

preprocess_data[100:300]

100%|██████████| 117997/117997 [04:22<00:00, 449.59it/s] 


[['아이템',
  '스크롤',
  '올려주다',
  '저항력',
  '올려주다',
  '올려주다',
  '투명하다',
  '해주다',
  '바르다',
  '해주다',
  '어렵다',
  '해주다',
  '괜찮다',
  '해주다',
  '스크롤',
  '순간이동',
  '순간이동',
  '스크롤'],
 [],
 ['평범하다',
  '아이템',
  '캐릭터',
  '강화하다',
  '싸우다',
  '입히다',
  '난이도',
  '아니다',
  '플레이어',
  '따르다',
  '가까이',
  '따라오다',
  '에서는',
  '완전하다',
  '벗어나다',
  '쫓아오다',
  '플레이어',
  '바뀌다',
  '조금씩',
  '안전하다',
  '레이스',
  '고스트'],
 ['상당하다',
  '강력하다',
  '상대로',
  '아니다',
  '불가능하다',
  '조금씩',
  '아나가야',
  '몰리다',
  '난이도',
  '안되다',
  '어리다',
  '따르다',
  '나가다',
  '저항력',
  '늘리다',
  '저항력',
  '올려주다',
  '저항력',
  '저항력',
  '챙기다'],
 ['숨기다',
  '경험치',
  '올리다',
  '어리다',
  '맞추다',
  '네가지',
  '채우다',
  '마지막',
  '아니다',
  '파란색',
  '나오다',
  '나오다',
  '이렇다',
  '맞추다',
  '해주다',
  '대부분',
  '숨기다',
  '가능하다',
  '비밀방',
  '퀘스트'],
 [],
 ['에서의', '자유롭다', '아니다', '마법사', '순간이동', '비싸다', '걸리다', '자유롭다'],
 ['마법사',
  '스크롤',
  '순간이동',
  '스크롤',
  '스크롤',
  '그리다',
  '비싸다',
  '가깝다',
  '순간이동',
  '순간이동',
  '스크롤',
  '비싸다',
  '괜찮다',
  '마법사',
  '순간이동',
  '배우다'],
 ['열리다', '으로의', '순간이동', '순간이동', '

In [7]:
nouns_per_sentence = [
    [w for w,t in okt.pos(" ".join(s), norm=True, stem=True) if t=='Noun']
    for s in preprocess_data
]
nouns_per_sentence = [ns for ns in nouns_per_sentence if ns]  # 빈 리스트 제거

In [8]:
nouns_per_sentence[:10]

[['에일리언', '컨테인먼트'],
 ['에이리언', '우주선', '동강이', '버스터', '줄거리', '스포일러', '혼수상태', '산소마스크', '생채기', '난장판'],
 ['컨테인먼트', '에이리언', '커버', '넌트'],
 ['롤플레잉', '모바일'],
 ['페이지'],
 ['모바일', '체험판', '성직자', '마법사', '플레이', '퀘스트', '퀘스트', '퀘스트'],
 ['체험판', '컨텐츠', '제작자'],
 ['플레이'],
 ['스토리'],
 ['식민지']]

In [9]:
len(nouns_per_sentence)

74709

In [ ]:
from gensim.models import FastText

wiki_model = FastText(        
    sentences=nouns_per_sentence, 
    vector_size=300, 
    window=5, 
    min_count=10, 
    sg=0
)

In [11]:
wiki_model.wv.vectors.shape

(5768, 300)

In [12]:
wiki_model.wv.most_similar('마법사')

[('사냥꾼', 0.9915006160736084),
 ('클래스', 0.988072395324707),
 ('카운터', 0.9865272045135498),
 ('크리티컬', 0.983668327331543),
 ('보너스', 0.9812778234481812),
 ('필살기', 0.9798440933227539),
 ('카오스', 0.976496696472168),
 ('코스트', 0.9734112620353699),
 ('딜', 0.9719274044036865),
 ('언데드', 0.9717555642127991)]

(10341, 100)

In [13]:
import random
from gensim.models import FastText

model = wiki_model  # 이미 학습된 FastText 모델 객체를 사용한다고 가정

# ---------------------------
# 유틸 함수들
# ---------------------------
def is_in_vocab(w):
    return w in model.wv.key_to_index

def random_word(exclude=set(), min_len=2):
    keys = model.wv.key_to_index.keys()
    while True:
        w = random.choice(list(keys))
        if (w not in exclude) and (len(w) >= min_len):
            return w

def sample_pair_with_similarity(threshold=0.8, max_tries=5000):
    """
    유사도 >= threshold인 (A,B) 한 쌍을 무작위로 뽑는다.
    """
    keys = list(model.wv.key_to_index.keys())
    for _ in range(max_tries):
        A = random.choice(keys)
        B = random.choice(keys)
        if A == B:
            continue
        sim = model.wv.similarity(A, B)
        if sim >= threshold:
            return A, B, sim
    raise RuntimeError(f"유사도>={threshold}인 쌍을 찾지 못했습니다. (max_tries={max_tries}): threshold를 낮춰보세요.")

def sample_C(exclude, max_tries=2000):
    """
    A,B와 너무 겹치지 않는 C를 뽑는다.
    """
    keys = list(model.wv.key_to_index.keys())
    for _ in range(max_tries):
        C = random.choice(keys)
        if C not in exclude and len(C) >= 2:
            return C
    raise RuntimeError("C를 뽑지 못했습니다.")

def compute_D_candidates(A, B, C, topn=10):
    """
    벡터 연산 B - A + C로 D 후보들을 구한다.
    A,B,C 자체는 제외.
    """
    candidates = model.wv.most_similar(positive=[B, C], negative=[A], topn=topn)
    filtered = [(w, s) for w, s in candidates if w not in {A, B, C}]
    return filtered

# ---------------------------
# 게임 본체
# ---------------------------
def play_analogy_game(rounds=5, sim_threshold=0.8, topn=10):
    print("== 단어 유추 게임 시작 ==")
    print("형식:  A : B  =  C : D  → D를 맞혀보세요!\n")
    score = 0

    for r in range(1, rounds+1):
        # 1) (A,B) 샘플링 (유사도 >= threshold)
        try:
            A, B, ab_sim = sample_pair_with_similarity(threshold=sim_threshold)
        except RuntimeError as e:
            print(e)
            break

        # 2) C 샘플링
        C = sample_C(exclude={A, B})

        # 3) D 후보 계산
        D_candidates = compute_D_candidates(A, B, C, topn=topn)
        if not D_candidates:
            # 후보가 없다면 라운드 재시도
            # (데이터가 작거나 희귀어가 많을 때 발생 가능)
            continue
        D_true, D_score = D_candidates[0]  # 정답 후보 1위

        # 4) 문제 제시
        print(f"[Round {r}]  {A} : {B}  =  {C} : ?")
        guess = input("당신의 답 D: ").strip()

        # 5) 판정
        correct_set = {w for w, _ in D_candidates[:3]}  # 상위 3개까지 정답 인정(난이도 조절)
        if guess in correct_set:
            print(f"✅ 정답! (정답 후보: {D_true}, 상위 후보: {', '.join(correct_set)})\n")
            score += 1
        else:
            print(f"❌ 오답!  정답 후보: {D_true}")
            print(f"   상위 후보들: {', '.join([w for w,_ in D_candidates[:5]])}\n")

    print(f"== 게임 종료: 점수 {score} / {rounds} ==")



In [14]:
# 실행
play_analogy_game(rounds=5, sim_threshold=0.8, topn=10)

== 단어 유추 게임 시작 ==
형식:  A : B  =  C : D  → D를 맞혀보세요!

[Round 1]  플레인즈 : 하이엔드  =  이등병 : ?
❌ 오답!  정답 후보: 학교폭력
   상위 후보들: 학교폭력, 공화국, 공무원, 일본군, 모스크바

[Round 2]  리자드 : 오스만  =  참고자료 : ?
❌ 오답!  정답 후보: 박물관
   상위 후보들: 박물관, 인민공화국, 위안스카이, 상하이, 기독교

[Round 3]  주거지역 : 조선왕조  =  필리스 : ?
❌ 오답!  정답 후보: 대혁명
   상위 후보들: 대혁명, 수뇌부, 유대인, 미하일, 군대장

[Round 4]  멜로디 : 텍사스  =  무면허 : ?
❌ 오답!  정답 후보: 서울시장
   상위 후보들: 서울시장, 중랑구, 교육감, 화성시, 선거인

[Round 5]  남학생 : 경상북도  =  스페인어 : ?
❌ 오답!  정답 후보: 스페인
   상위 후보들: 스페인, 미합중국, 아시아, 국가대표, 합중국

== 게임 종료: 점수 0 / 5 ==
